In [1]:
1

1

In [2]:
import pdfplumber
import json

def extract_ttas_tables(pdf_path):
    all_content = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 提取表格數據
            tables = page.extract_tables()
            # 提取當頁純文字（用於獲取標題與備註）
            text = page.extract_text()
            all_content.append({"text": text, "tables": tables})
    return all_content

In [8]:
file= extract_ttas_tables(r"C:\Users\ygz08\Work\TTAS\code\rag_files\急診五級檢傷分類基準修正版-環境.pdf")
file

[{'text': '急診五級檢傷分類基準修正版-環境\nTTAS 次要調\n大分類名稱 標準主訴名稱 判定依據中文名稱 編碼\n檢傷級數 節變數\nE01環境 E0101昆蟲螫傷 重度呼吸窘迫(<90%) 1 E010101\n休克 1 E010102\n無意識(GCS3-8) 1 E010103\n中度呼吸窘迫(<92%) 2 E010104\n血行動力循環不足 2 E010105\n意識程度改變(GCS9-13) 2 E010106\n過去曾出現嚴重過敏反應 2 ＊ E010107\n臉部、嘴唇與舌頭腫脹 2 ＊ E010108\n輕度呼吸窘迫(92-94%) 3 E010109\n血壓或心跳有異於病人之平常數值，但血行動力穩定 3 E010110\n急性周邊重度疼痛(8-10) 3 E010111\n廣泛性紅疹/水泡 3 ＊ E010112\n急性周邊中度疼痛(4-7) 4 E010113\n局部性紅疹/水泡(過敏反應) 4 ＊ E010114\n急性周邊輕度疼痛(<4) 5 E010115\n輕微螫傷，生命徵象正常，無發燒，無疼痛不適 5 ＊ E010116\nE0102海洋生物螫傷 重度呼吸窘迫(<90%) 1 E010201\n休克 1 E010202\n無意識(GCS3-8) 1 E010203\n中度呼吸窘迫(<92%) 2 E010204\n血行動力循環不足 2 E010205\n意識程度改變(GCS9-13) 2 E010206\n過去曾出現嚴重過敏反應 2 ＊ E010207\n臉部、嘴唇與舌頭腫脹 2 ＊ E010208\n輕度呼吸窘迫(92-94%) 3 E010209\n血壓或心跳有異於病人之平常數值，但血行動力穩定 3 E010210\n急性周邊重度疼痛(8-10) 3 E010211\n廣泛性紅疹/水泡 3 ＊ E010212\n急性周邊中度疼痛(4-7) 4 E010213\n局部性紅疹/水泡(過敏反應) 4 ＊ E010214\n急性周邊輕度疼痛(<4) 5 E010215\n輕微螫傷，生命徵象正常，無發燒，無疼痛不適 5 ＊ E010216\nE0103動物咬傷 重度呼吸窘迫(<90%) 1 E010301\n休克 1 E010302\n無意識(GCS3-8) 1 E010303\n中度呼吸窘迫(<92%) 2 E010304

'急診五級檢傷分類基準修正版-環境'

In [ ]:
#file[0] # page 0
#print(file[0]["tables"])# page 0 的table
#print(file[0]["tables"][0][0]) # 第一row #標題
#print(file[0]["tables"][0][1]) # 第二row ，第一筆正式資料。
#print(file[0]["tables"][0][1][0]) # 大分類名稱
#print(file[0]["tables"][0][1][1]) # 標準主訴名稱 != None 代表在同一個主訴內 
print(file[0]["tables"][0][1][2]) #判定依據中文名稱
print(file[0]["tables"][0][1][3]) # TTAS 分級

重度呼吸窘迫(<90%)
1


In [ ]:
dict1 = {}
dict1["metadata"] = {}
dict1["content"] = ""
dict1["remarks"] = ""
#檔案層級:
category = file[0]["text"].split('\n')[0]  #檔案名稱
dict1["metadata"]["category"] = category
target = "成人/兒童" #"成人" # "兒童"        ##TODO:手動決定
dict1["metadata"]["target"] = target

standard_name = 1 # 標準主訴名稱
determined_name = 2 #判定依據中文名稱
TTAS_level = 3 # TTAS等級
adjust = 4 #次要調節變數

for row in range(1, len(file[0]["tables"][0])): #每行
    if file[0]["tables"][0][1][0] != None:                          # 大分類名稱
        top_category_name = file[0]["tables"][0][1][0]
        dict1["metadata"]["sub_category"] = top_category_name

    if file[0]["tables"][0][row][standard_name] != None:             # 標準主訴名稱
        #print(file[0]["tables"][0][row][standard_name])  #所有的標準主訴名稱
        now_name = file[0]["tables"][0][row][standard_name]
        dict1["metadata"]["query_text"] = now_name
        #refresh level_one~level_five
        level_one, level_two, level_three, level_four, level_five, = "一級 (復甦急救)：", "二級 (危急)：", "三級 (緊急)：", "四級 (次緊急)：", "五級 (非緊急)："
        adjust_str = "可調節分級項目 : "

    # 子項，標準主訴名稱 為 now_name。
    #判定依據中文名稱 處理
    subtitle = file[0]["tables"][0][row][determined_name] # get每一行都有的判定依據中文名稱


    if file[0]["tables"][0][row][TTAS_level] == "1":                 #TTAS 1
        level_one += subtitle
        level_one += "。"

    if file[0]["tables"][0][row][TTAS_level] == "2":                 # TTAS 2
        level_two += subtitle
        level_two += "。"

    if file[0]["tables"][0][row][TTAS_level] == "3":                 # TTAS 3 
        level_three += subtitle
        level_three += "。"

    if file[0]["tables"][0][row][TTAS_level] == "4":                 # TTAS 4
        level_four += subtitle
        level_four += "。"
 
    if file[0]["tables"][0][row][TTAS_level] == "5":                 # TTAS 5
        level_five += subtitle
        level_five += "。"

    if file[0]["tables"][0][row][adjust] == "＊":                 # 次要調節變數
        adjust_str += subtitle
        adjust_str += "。"


    #最後統整
    dict1["content"] = "- " + level_one + "\n- " + level_two + "\n- " + level_three + "\n- " + level_four + "\n- " + level_five
    dict1["remarks"] = adjust_str

#TODO:1: 更新all_dict 裝dict1，讓每個1u痾沝主訴都可呈現。
#TODO:2: 當level_five 沒有出現時，要擋住+= subtitle。

In [53]:
formatted_json = json.dumps(dict1, indent=4, ensure_ascii=False)

print(formatted_json)

{
    "metadata": {
        "category": "急診五級檢傷分類基準修正版-環境",
        "target": "成人/兒童",
        "sub_category": "E01環境",
        "query_text": "E0103動物咬傷"
    },
    "content": "- 一級 (復甦急救)：重度呼吸窘迫(<90%)。休克。無意識(GCS3-8)。\n- 二級 (危急)：中度呼吸窘迫(<92%)。血行動力循環不足。意識程度改變(GCS9-13)。凝血異常-危及生命或肢體的出血。無法控制的出血。\n- 三級 (緊急)：輕度呼吸窘迫(92-94%)。血壓或心跳有異於病人之平常數值，但血行動力穩定。急性周邊重度疼痛(8-10)。凝血異常-中度或輕度的出血。加壓後已止血。\n- 四級 (次緊急)：急性周邊中度疼痛(4-7)。\n- 五級 (非緊急)：",
    "remarks": "可調節分級項目 : 凝血異常-危及生命或肢體的出血。無法控制的出血。凝血異常-中度或輕度的出血。加壓後已止血。"
}


In [ ]:

dict1["metadata"] = {}
dict1["metadata"]["category"] = category
dict1["metadata"]["sub_category"] = top_category_name
dict1["metadata"]["target"] = 
dict1["metadata"]["source_page"] = 
dict1["metadata"]["query_text"] = now_name


{'metadata': {'category': '急診五級檢傷分類基準修正版-環境'}}


{'metadata': {'category': '急診五級檢傷分類基準修正版-環境'}}

In [ ]:
trauma_t1201_chunk = {
    "metadata": {
        "category": "急診五級檢傷分類基準修正版-外傷",
        "sub_category": "T12 肢體外傷",
        "target": "成人/兒童",
        "source_page": 10,
        "query_text": "T1201 上肢鈍傷"
    },
    "content": "二級 (危急):急性周邊重度疼痛(8-10)，高危險性受傷機轉，凝血異常-危及生命或肢體的出血，神經受損或肢體血管循環不佳徵象，三級 (緊急):急性周邊中度疼痛(4-7)，凝血異常-中度或輕度的出血，腫脹變形,疑似骨折/脫臼，五級 (非緊急):軟組織挫傷/血腫",
    "remarks": "凝血異常-危及生命或肢體的出血、神經受損或肢體血管循環不佳徵象、凝血異常-中度或輕度的出血、腫脹變形,疑似骨折/脫臼、軟組織挫傷/血腫"
}